In [ ]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import fisher_exact
import shelve
import pickle
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
import csv
from dfply import *
import itertools
import os
import logging
from sys import getsizeof
import sklearn
import time
from sklearn.metrics import roc_auc_score
from catboost import Pool, cv
import xgboost
import catboost
import scipy.stats as st

import importlib
from joblib import Parallel, delayed
from joblib import parallel_backend

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.stattools import adfuller, kpss, grangercausalitytests, pacf
from statsmodels.graphics.tsaplots import plot_pacf
import warnings
warnings.filterwarnings("ignore")
import scipy
import datetime.timedelt

In [ ]:
site = 'UTHSCSA'
datafolder = '/home/hoyinchan/blue/Data/data2021raw/'
lab = pd.read_csv(datafolder+site+'/raw/AKI_LAB.csv')
onset = pd.read_csv(datafolder+site+'/raw/AKI_ONSETS.csv') 
onset = onset[onset['NONAKI_SINCE_ADMIT'].isnull()]
labx = lab[(lab['LAB_LOINC']=='2160-0') | (lab['LAB_LOINC']=='17861-6')]
labx = labx[['PATID','ENCOUNTERID','SPECIMEN_DATE_TIME','LAB_LOINC','RESULT_NUM']]
labx['SPECIMEN_DATE_TIME'] = pd.to_datetime(labx['SPECIMEN_DATE_TIME'])
labx = labx.groupby(['PATID','ENCOUNTERID','SPECIMEN_DATE_TIME','LAB_LOINC']).mean().reset_index()
labxx = labx.pivot(index=['PATID','ENCOUNTERID','SPECIMEN_DATE_TIME'], columns = 'LAB_LOINC', values='RESULT_NUM').reset_index()
data_count = labxx.groupby(['PATID','ENCOUNTERID']).count().sort_values('SPECIMEN_DATE_TIME').reset_index()

In [ ]:
onset = onset[['PATID','ENCOUNTERID', '', 'AKI1_SINCE_ADMIT', 'AKI2_SINCE_ADMIT', 'AKI3_SINCE_ADMIT']].min(axis=1)

In [ ]:
onset

In [ ]:
data_count.tail(10)

In [ ]:
tmp = labxx[labxx['ENCOUNTERID']==58476972183]

In [ ]:
data_count['ENCOUNTERID'].iloc[-1]

In [ ]:
tmp.index=tmp['SPECIMEN_DATE_TIME']

In [ ]:
s = tmp['17861-6']
r = tmp['2160-0']

In [ ]:
td = (np.diff(s.index).mean().astype('timedelta64[h]')*1).astype(int)
td = str(td)+'H'

In [ ]:
def regular_ts(s):
    oidx = s.index
    nidx = pd.date_range(oidx.min(), oidx.max(), freq=td)
    res = s.reindex(oidx.union(nidx)).interpolate('index').reindex(nidx)
    res.plot(style='.-')
    s.plot(style='o')    
    return res

In [ ]:
s = regular_ts(s)

In [ ]:
r = regular_ts(r)

In [ ]:
tentmap_df = pd.DataFrame([s, r]).T
tentmap_df = tentmap_df.reset_index(drop=True).reset_index()
tentmap_df.columns = ['Time','calcium', 'scr']

In [ ]:
tentmap_df.to_parquet('tentmap_df.parquet')

In [ ]:
plt.scatter(x=s.index, y=s)

In [ ]:
testdf = pd.DataFrame([0,1,2,3,4,5,6,7,8,9,10])

In [ ]:
testdf = testdf.reset_index()

In [ ]:
testdf.columns = ['Time', 'x']

In [ ]:
pyEDM.Embed(dataFrame=testdf, columns='x', E=4, tau=-2)

In [ ]:
import pyEDM

In [ ]:
optimal_emdeddings = list()
lx = tentmap_df.shape[0]
lib = "1 "+str(lx)
pred = lib
for tau in range(-10,0):
    for E in range(1,10):
        try:
            x1 = pyEDM.Simplex(dataFrame=tentmap_df, lib=lib , pred=pred, columns='calcium', target='calcium', Tp=1, tau=tau, E=E)
            optimal_emdedding = pd.DataFrame([E, pyEDM.ComputeError(x1['Observations'], x1['Predictions'])['rho']]).T
            optimal_emdedding.columns = ['E', 'rho']
#            optimal_emdedding = pyEDM.EmbedDimension(dataFrame=tentmap_df, lib=lib , pred=pred, columns='calcium', target='calcium', Tp=1, tau=tau, maxE=7)
            optimal_emdedding['tau'] = tau
            optimal_emdedding['target'] = 'calcium'
            optimal_emdeddings.append(optimal_emdedding)        
            
            x1 = pyEDM.Simplex(dataFrame=tentmap_df, lib=lib , pred=pred, columns='scr', target='scr', Tp=1, tau=tau, E=E)
            optimal_emdedding = pd.DataFrame([E, pyEDM.ComputeError(x1['Observations'], x1['Predictions'])['rho']]).T
            optimal_emdedding.columns = ['E', 'rho']            
#            optimal_emdedding = pyEDM.EmbedDimension(dataFrame=tentmap_df, lib=lib , pred=pred, columns='scr', target='scr', Tp=1, tau=tau, maxE=7)
            optimal_emdedding['tau'] = tau     
            optimal_emdedding['target'] = 'scr'        
            optimal_emdeddings.append(optimal_emdedding)
        except:
            pass
optimal_emdeddings = pd.concat(optimal_emdeddings)

In [ ]:
optimal_values = optimal_emdeddings.sort_values('tau',ascending=False).groupby('target').head(1)

In [ ]:
#libSizes = ','.join(str(x) for x in list(range(5, len(tentmap_df),5)))
libSizes=str(len(tentmap_df)-1)

In [ ]:
step = 5
sample = 10
libSizes = ' '.join([str(x) for x in [5, tdf.shape[0], 5]])

In [ ]:
r1 = pyEDM.CCM(dataFrame=tdf, columns='calcium', target='scr', E=2, Tp=1, tau=-1, sample = 10, libSizes=libSizes, replacement=True, includeData=True)
r2 = pyEDM.CCM(dataFrame=tdf, columns='scr', target='calcium', E=1, Tp=1, tau=-1, sample = 10, libSizes=libSizes, replacement=True, includeData=True)

In [ ]:
r1['PredictStats1']

In [ ]:
def ttest(a):
    return scipy.stats.ttest_1samp(a, 0).pvalue

In [ ]:
r1p = r1['PredictStats1'][['E','nn','tau','LibSize','rho']].groupby(['E','nn','tau','LibSize']).agg([np.mean, np.std, ttest]).reset_index()
r2p = r2['PredictStats1'][['E','nn','tau','LibSize','rho']].groupby(['E','nn','tau','LibSize']).agg([np.mean, np.std, ttest]).reset_index()

In [ ]:
plt.plot(r1p['LibSize'], r1p[('rho','mean')])
plt.fill_between(r1p['LibSize'], r1p[('rho','mean')]-2*r1p[('rho','std')], r1p[('rho','mean')]+2*r1p[('rho','std')], alpha=0.1)
plt.plot(r2p['LibSize'], r2p[('rho','mean')])
plt.fill_between(r2p['LibSize'], r2p[('rho','mean')]-2*r2p[('rho','std')], r2p[('rho','mean')]+2*r2p[('rho','std')], alpha=0.1)
plt.axhline(y=0, color='g')
plt.show()

In [ ]:
Tps = list(range(-10,10))
rphos = list()
for Tp in Tps: 
    tdf = tentmap_df.copy()
    sample = 200    
#    libSizes=str(len(tdf))
    libSizes=str(50)
    rpho = pyEDM.CCM(dataFrame=tdf, columns='calcium', target='scr', E=4, Tp=Tp, tau=-1, sample = sample, libSizes=libSizes, replacement=True, includeData=True)
    rpho['Tp'] = Tp
    rphos.append(rpho)
rphodf = pd.concat([x['LibMeans'] for x in rphos])
rphodf['std'] = [x['PredictStats1']['rho'].std() for x in rphos]
rphodf['Tp'] = [x['Tp'] for x in rphos]

In [ ]:
plt.plot(rphodf['Tp'], rphodf['calcium:scr'])
plt.fill_between(rphodf['Tp'], rphodf['calcium:scr']-rphodf['std'], rphodf['calcium:scr']+rphodf['std'],alpha=0.1)
plt.axvline(x=0, color='r')
plt.axhline(y=0, color='g')

In [ ]:
Tps = list(range(-10,10))
rphos = list()
bootstrap = 10
for Tp in Tps: 
    tdf = tentmap_df.copy()
    sample = 100
#    libSizes=str(len(tdf))    
    libSizes=str(50)    
    rpho = pyEDM.CCM(dataFrame=tdf, columns='scr', target='calcium', E = 1, Tp=Tp, tau=-1, sample = 100, libSizes=libSizes, replacement=True, includeData=True)
    rpho['Tp'] = Tp
    rphos.append(rpho)
rphodf = pd.concat([x['LibMeans'] for x in rphos])
rphodf['std'] = [x['PredictStats1']['rho'].std() for x in rphos]
rphodf['Tp'] = [x['Tp'] for x in rphos]

In [ ]:
plt.plot(rphodf['Tp'], rphodf['scr:calcium'])
plt.fill_between(rphodf['Tp'], rphodf['scr:calcium']-rphodf['std'], rphodf['scr:calcium']+rphodf['std'],alpha=0.1)
plt.axvline(x=0, color='r')
plt.axhline(y=0, color='g')

In [ ]:
import myccm